# analysis pipeline for psd-slope

This notebook covers the whole analysis for the psd-slope project.

##### Jorge Yanar 2016
This notebook does the following:
1. Imports every .mat file in the specified directory
2. For each file, we run through channels and compute the PSD of each channel
3. Compute the average PSD for each subject
4. Use the RANSAC algorithm to fit a line to each PSD, using the PSD from 1 - 24 Hz but blocking out alpha, from 7 - 14 Hz.

#### Imports

In [21]:
%matplotlib inline
import os
import glob
import numpy as np
import scipy as sp
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import signal
from scipy import io
from sklearn import linear_model, datasets

#### Data importing

In [22]:
subj = {} # Dictionary that holds all subjects
matfiles = []
for root, dirs, files in os.walk('../data/extClfiltCAR-mat/'):
    matfiles += glob.glob(os.path.join(root, '*.mat'))

for i in range(len(matfiles)):
    datafile = sp.io.loadmat(matfiles[i])
    
    subj[i] = {}
    subj[i]['name']  = np.squeeze(datafile['name'])
    subj[i]['srate'] = np.squeeze(datafile['srate'])
    subj[i]['data']  = np.squeeze(datafile['data'])    

#### Compute PSD

In [43]:
def compute_slope(f, psd):
    """
    Computes PSD and fits line to it, calculating the slope.
    
    Output:
        slope
    """
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(f[:50*2], np.log10(psd[:50*2]))
    fit_line = model_ransac.predict(f)
    return model_ransac.estimator_.coef_[0][0]

Now to compute the slope of every channel for each subject:

In [46]:
srate = 512
for i in subj:
    for ch in range(len(subj[i]['data'])):
        subj[i][ch] = {}
        
        f, psd = sp.signal.welch(subj[i]['data'][ch], srate, nperseg=srate*2, noverlap=srate, window='hamming')
        f   = f.reshape((len(f), 1))
        psd = psd.reshape((len(psd), 1))
        
        subj[i][ch]['f'] = f
        subj[i][ch]['psd'] = psd
        subj[i][ch]['slope'] = compute_slope(f, psd)

And find the average slope for each subject.

In [47]:
for i in subj:
    slopes = []
    for ch in range(len(subj[i]['data'])):
        slopes.append(subj[i][ch]['slope'])
    subj[i]['slope'] = np.mean(slopes)

In [59]:
for i in subj:
    print("{}: {:.4f}".format(subj[i]['name'], subj[i]['slope']))

120127101: -0.0514
120127102: -0.0515
120127103: -0.0261
120127104: -0.0491
120127105: -0.0390
120127106: -0.0224
120127107: -0.0299
120127108: -0.0360
120127109: -0.0287
120127110: -0.0260
120127111: -0.0367
120127112: -0.0487
120127113: -0.0401
120127114: -0.0313
120127115: -0.0324
120127116: -0.0391
120127117: -0.0246
120127118: -0.0472
120127119: -0.0205
120127120: -0.0335
120127121: -0.0044
120127122: -0.0356
120127123: -0.0439
120127124: -0.0302
120127125: -0.0512
120127128: -0.0512
120127130: -0.0210
120127131: -0.0366
120127132: -0.0389
120127133: -0.0445
120127134: -0.0382
120127135: -0.0455
120127137: -0.0397
120127138: -0.0013
120127139: -0.0354
120127140: -0.0397
120127142: -0.0432
120127144: -0.0162
120127145: -0.0290
120127146: -0.0499
120127147: -0.0245
120127148: -0.0385
120127149: -0.0472
120127151: -0.0103
120127153: -0.0230
120127154: -0.0238
120127155: -0.0189
120127156: -0.0389
120127157: -0.0480
120127158: -0.0475
120127159: -0.0365
120127160: -0.0381
120127161: -

#### Export results to CSV